In [5]:
import tensorflow as tf

x = tf.Variable(0, name='counter')
assign_op = tf.assign(x, 0)
p = tf.placeholder(tf.int32, ())
with tf.control_dependencies([assign_op]):
    x = tf.assign_add(x, p, name='increment')
    x = tf.assign_add(x, p, name='increment_2')

with tf.Session() as sess:
    # writer = tf.summary.FileWriter('logs', sess.graph)
    sess.run(tf.global_variables_initializer())
    for _ in range(5):
        print(sess.run(x, {p: 1}))

2
2
2
2
2
